In [ ]:
def nfs_setup(nfs_client_ip="172.16.27.100/16", nfs_server_ip="172.16.27.175"):  # IP 변경 필요
    """
    Ubuntu NFS 서버 설치, 공유 디렉토리 생성 및 클라이언트 마운트
    nfs_server_ip : 서버로 사용할 IP
    nfs_client_ip : 클라이언트에서 접근 허용할 IP (예: 172.16.27.175/24)
    """
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, password=pw, port=22)

    def run_remote(cmd, show=True):
        stdin, stdout, stderr = ssh.exec_command(cmd)
        for line in iter(stdout.readline, ""):
            if show:
                print(line, end="")
        err = stderr.read().decode()
        if err and show:
            print("[ERROR]", err)
        return err

    print("=== Ubuntu NFS 서버 설치 및 공유 디렉토리 설정 ===")
    run_remote("apt update -y")
    run_remote("apt install -y nfs-kernel-server nfs-common")

    # 공유 디렉토리 생성
    run_remote("mkdir -p /mnt/nfs_share")
    run_remote("chown nobody:nogroup /mnt/nfs_share")
    run_remote("chmod 777 /mnt/nfs_share")

    # /etc/exports 설정
    run_remote(f'echo "/mnt/nfs_share {nfs_client_ip}(rw,sync,no_root_squash)" > /etc/exports')

    # 서비스 활성화
    run_remote("systemctl enable nfs-kernel-server")
    run_remote("systemctl restart nfs-kernel-server")
    run_remote("exportfs -v")  # 공유 확인
    print(" NFS 서버 설정 완료")

    # 클라이언트 마운트
    print("=== NFS 클라이언트 마운트 ===")
    run_remote("apt install -y nfs-common")
    run_remote("mkdir -p /mnt/nfs_data")
    run_remote(f"mount -t nfs {nfs_server_ip}:/mnt/nfs_share /mnt/nfs_data")
    print(" NFS 클라이언트 마운트 완료")

    ssh.close()